In [22]:
import brian2 as br
import numpy as np
import pandas as pd
import repro_functions as repro

import scipy.io as sio

In [23]:
def load_data(group):

    #fol = r'f:\desktop\Code\Reproducibility\model_ICx\SynchResults\data\iccl'
    fol = r'K:\python\model_ICx_codeonly\SynchResults\data\iccl'
    #group = -180

    dataset = sio.loadmat(fol + '\\SynchICcl_ITD' + str(group) + '_001_SNR20.mat')
    spikes = dataset['spikesICcl']

    del fol, dataset
    
    return spikes

In [24]:
def set_up(spikes):
    dur = 0.3
    cw = 0.0001 * br.second
    bins = len(np.arange(0 * br.second, dur * br.second + 0.05 * br.second + cw, cw)) - 1
    neurons, itd, reps = np.shape(spikes)
    return dur, cw, bins, neurons, itd, reps

In [25]:
def make_hist(spikes, neurons, itd, bins):

    hist = np.ndarray([int(neurons), int(itd), bins])

    for t in range(itd):
        for n in range(neurons):
            sp = spikes[n, t, :]
            sp = [s/1000 for s in sp]
            try:
                hist[n, t, :] = repro.make_PSTH(np.concatenate(sp, 1), dur * br.second, cw, reps, 0 * br.second)
            except:
                print(str(t), str(n))

    del t, n, sp

    return hist

In [26]:
def run_SAC(itd, neurons, hist, reps, cw):

    # variables
    curve_time = 100

    rep_out = pd.DataFrame(columns = ['neuron', 'itd', 'fr', 'curve'])

    for t in range(itd):
        for n in range(neurons):
            try:
                e, SAC = repro.autocor(hist[n, t, :] * br.Hz, reps, T=curve_time*br.ms, bin=cw)
                SAC = SAC * br.Hz ** 2
                sp = np.mean(hist[n, t, :])
            except:
                print(str(t), str(n))
                arr = np.arange(-(curve_time * br.ms), (curve_time * br.ms) , cw)
                arr = np.delete(arr, 0)
                arr[:] = np.nan
                SAC = arr
                del arr
                sp = np.nan
                pass
            tmp = {'neuron': n, 'itd': t, 'fr':sp, 'curve': SAC}
            rep_out = rep_out.append(tmp, ignore_index = True)

    del e, n, t, SAC, sp, tmp

    return rep_out

In [27]:
def save_data(rep_out, group):

    curve = [c for c in rep_out['curve']]

    sio.savemat('K:\\python\\reproducibility\\data\\iccl\\repro_iccl_ITD' + str(group) + '_001_SNR20.mat', {'curve':curve, 'cw':cw, 'neuron':rep_out['neuron'].tolist(), 'itd':rep_out['itd'].tolist(), 'fr':rep_out['fr'].tolist(), 'reps':reps, 'dur':dur})

    del curve

In [20]:
del bins, curve_time, cw, dur, group, neurons, reps, hist, spikes, rep_out

NameError: name 'curve_time' is not defined

In [32]:
groups = np.arange(-200, -20, 20)

In [34]:
#for g in groups:
    
g = 200
print('running ' + str(g))
print('loading')
spikes = load_data(g)
print('set up')
dur, cw, bins, neurons, itd, reps = set_up(spikes)
print('making hist')
hist = make_hist(spikes, neurons, itd, bins)
print('running sac')
rep_out = run_SAC(itd, neurons, hist, reps, cw)
print('saving')
save_data(rep_out, g)
print('')

running 200
loading
set up
making hist
running sac
saving

